In [44]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
import optuna
import xgboost
from xgboost import XGBClassifier
import lightgbm 
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier


In [81]:
# csv file 불러오기
train = pd.read_csv('/Users/euijinlee/Downloads/open/train.csv')
test = pd.read_csv('/Users/euijinlee/Downloads/open/test.csv')

# 상위 세줄 불러오기
train.head(3)

,ID,generation,school1,major type,major1_1,major1_2,major_data,job,class1,class2,...,incumbents_company_level,incumbents_lecture_type,incumbents_lecture_scale,incumbents_lecture_scale_reason,interested_company,expected_domain,contest_participation,idea_contest,onedayclass_topic,completed
0,TRAIN_000,9,22,"복수 전공 ( 다중전공, 이중전공 포함 )",경제통상학,자연과학,False,대학생,1,4.0,...,해외 기업 (빅테크),"온, 오프라인 동시",100명 이상의 리스너와 10명 이상의 현직자,다양한 사람들과 만나서 생각을 교류할 수 있기 때문,"구글 딥마인드, 카카오 브레인","M. 전문, 과학 및 기술 서비스업",NaN,NaN,"Python 응용, 데이터 시각화 (Matplotlib, Seaborn 등), 머신...",0
1,TRAIN_001,9,1,"복수 전공 ( 다중전공, 이중전공 포함 )",자연과학,IT(컴퓨터 공학 포함),True,대학생,8,NaN,...,국내 빅테크 IT 계열 (네카라쿠배당토),오프라인,3~50명 내외의 강의 리스너와 1명의 현직자,더 많은 사람들이 있으면 제가 예상하지 못한 질문도 할 수 있다고 생각하기 때문입니다.,제일 기획,"J. 정보통신업, O. 공공 행정, 국방 및 사회보장 행정",NaN,NaN,머신러닝 / 딥러닝 응용,0
2,TRAIN_002,9,27,단일 전공,예체능,NaN,False,대학생,7,NaN,...,"국내 대기업 IT 계열 (금융, 제조 ...)",오프라인,3~50명 내외의 강의 리스너와 1명의 현직자,인원이 너무 적으면 서로 부담스러울 수 있을 것 같지만 너무 많으면 너무 피상적인 ...,Lg전자,"C. 제조업, K. 금융 및 보험업, R. 예술, 스포츠 및 여가관련 서비스업",NaN,NaN,"머신러닝 / 딥러닝 응용, SQL 응용, 웹 크롤링",0


In [46]:
# 데이터 정보(Null-count와 datatype) 불러오기
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 46 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   ID                               748 non-null    object 
 1   generation                       748 non-null    int64  
 2   school1                          748 non-null    int64  
 3   major type                       726 non-null    object 
 4   major1_1                         728 non-null    object 
 5   major1_2                         309 non-null    object 
 6   major_data                       748 non-null    bool   
 7   job                              748 non-null    object 
 8   class1                           748 non-null    int64  
 9   class2                           169 non-null    float64
 10  class3                           14 non-null     float64
 11  class4                           1 non-null      float64
 12  re_registration       

In [47]:
#데이터 상위 한줄 불러오기
train.head(1)

,ID,generation,school1,major type,major1_1,major1_2,major_data,job,class1,class2,...,incumbents_company_level,incumbents_lecture_type,incumbents_lecture_scale,incumbents_lecture_scale_reason,interested_company,expected_domain,contest_participation,idea_contest,onedayclass_topic,completed
0,TRAIN_000,9,22,"복수 전공 ( 다중전공, 이중전공 포함 )",경제통상학,자연과학,False,대학생,1,4.0,...,해외 기업 (빅테크),"온, 오프라인 동시",100명 이상의 리스너와 10명 이상의 현직자,다양한 사람들과 만나서 생각을 교류할 수 있기 때문,"구글 딥마인드, 카카오 브레인","M. 전문, 과학 및 기술 서비스업",NaN,NaN,"Python 응용, 데이터 시각화 (Matplotlib, Seaborn 등), 머신...",0


In [48]:
# 오브젝트 데이터값들을 가진 컬럼들 중에 20개 이하로 분류가능한 컬럼들 따로 저장하기

cate_usecol = []
temp = train.select_dtypes(include="object")
for col in temp.columns:
    if len(temp[col].unique()) <= 20:
        cate_usecol.append(col)

In [49]:
cate_usecol

['major type',
 'major1_1',
 'major1_2',
 'job',
 're_registration',
 'nationality',
 'inflow_route',
 'whyBDA',
 'what_to_gain',
 'hope_for_group',
 'previous_class_3',
 'previous_class_4',
 'previous_class_5',
 'previous_class_6',
 'previous_class_7',
 'previous_class_8',
 'desired_career_path',
 'project_type',
 'incumbents_level',
 'incumbents_lecture',
 'incumbents_company_level',
 'incumbents_lecture_type',
 'incumbents_lecture_scale',
 'contest_participation']

In [50]:
# 오브젝트값이 아닌 컬럼들 정보 불러오기

train.select_dtypes(exclude="object").info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   generation          748 non-null    int64  
 1   school1             748 non-null    int64  
 2   major_data          748 non-null    bool   
 3   class1              748 non-null    int64  
 4   class2              169 non-null    float64
 5   class3              14 non-null     float64
 6   class4              1 non-null      float64
 7   contest_award       0 non-null      float64
 8   completed_semester  720 non-null    float64
 9   time_input          748 non-null    float64
 10  idea_contest        0 non-null      float64
 11  completed           748 non-null    int64  
dtypes: bool(1), float64(7), int64(4)
memory usage: 65.1 KB


In [51]:
# 오브젝트값이 아닌 컬럼들 cont_usecol에 저장하기
# 숫자가 적거나 0인 컬럼은 제외 
cont_usecol = ['generation', 'school1', 'major_data', 'class1', 'class2', 'completed_semester', 'time_input', 'completed']


# 방금 저장한 컬럼들 정보만 불러오기
train[cont_usecol].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   generation          748 non-null    int64  
 1   school1             748 non-null    int64  
 2   major_data          748 non-null    bool   
 3   class1              748 non-null    int64  
 4   class2              169 non-null    float64
 5   completed_semester  720 non-null    float64
 6   time_input          748 non-null    float64
 7   completed           748 non-null    int64  
dtypes: bool(1), float64(3), int64(4)
memory usage: 41.8 KB


In [52]:
# 쓸 데이터들(cont_usecol와 cate_usecol)합쳐서 다시 저장하기
train = train[cont_usecol+cate_usecol]


In [53]:
from pycaret.classification import *


# pycaret으로 모델 성능 비교
setup_clf = setup(data=train, target='completed')
class_top = compare_models()
class_top

,Description,Value
0,Session id,7314
1,Target,completed
2,Target type,Binary
3,Original data shape,"(748, 32)"
4,Transformed data shape,"(748, 146)"
5,Transformed train set shape,"(523, 146)"
6,Transformed test set shape,"(225, 146)"
7,Numeric features,6
8,Categorical features,24
9,Rows with missing values,100.0%


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dummy,Dummy Classifier,0.7017,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0220
rf,Random Forest Classifier,0.6940,0.5393,0.0571,0.4917,0.1004,0.0294,0.0657,0.0360
catboost,CatBoost Classifier,0.6921,0.5429,0.0508,0.4333,0.0888,0.0212,0.0519,0.2460
ada,Ada Boost Classifier,0.6673,0.5456,0.2004,0.4458,0.2574,0.0777,0.0956,0.0310
ridge,Ridge Classifier,0.6652,0.5176,0.1608,0.4080,0.2227,0.0513,0.0690,0.0230
gbc,Gradient Boosting Classifier,0.6616,0.5380,0.0775,0.2739,0.1117,-0.0139,-0.0057,0.0350
knn,K Neighbors Classifier,0.6614,0.5272,0.2104,0.4434,0.2598,0.0728,0.0919,0.0470
lda,Linear Discriminant Analysis,0.6595,0.5100,0.1867,0.3898,0.2435,0.0563,0.0683,0.0430
lr,Logistic Regression,0.6576,0.5285,0.1479,0.4150,0.2048,0.0308,0.0535,0.0590
et,Extra Trees Classifier,0.6500,0.4902,0.0771,0.2310,0.1104,-0.0364,-0.0466,0.0400


DummyClassifier(constant=None, random_state=7314, strategy='prior')

In [54]:
# xgboost, lightgbm, naivebayes 

In [55]:
from sklearn.naive_bayes import GaussianNB

#Create a Gaussian Classifier
#model = GaussianNB()

# Train the model using the training sets
#model.fit(features,label)

In [82]:
# 학습용 데이터랑 테스트용 데이터에서 둘다 쓸 독립변수 컬럼들만 뽑아서 저장

X = train[['generation', 'school1', 'major_data', 'class1', 'class2',
       'completed_semester', 'time_input', 'major type',
       'major1_1', 'major1_2', 'job', 're_registration', 'nationality',
       'inflow_route', 'whyBDA', 'what_to_gain', 'hope_for_group',
       'previous_class_3', 'previous_class_4', 'previous_class_5',
       'previous_class_6', 'previous_class_7', 'previous_class_8',
       'desired_career_path', 'project_type', 'incumbents_level',
       'incumbents_lecture', 'incumbents_company_level',
       'incumbents_lecture_type', 'incumbents_lecture_scale']]

X_test = test[['generation', 'school1', 'major_data', 'class1', 'class2',
              'completed_semester', 'time_input', 'major type',
              'major1_1', 'major1_2', 'job', 're_registration', 'nationality',
              'inflow_route', 'whyBDA', 'what_to_gain', 'hope_for_group',
              'previous_class_3', 'previous_class_4', 'previous_class_5',
              'previous_class_6', 'previous_class_7', 'previous_class_8',
              'desired_career_path', 'project_type', 'incumbents_level',
              'incumbents_lecture', 'incumbents_company_level',
              'incumbents_lecture_type', 'incumbents_lecture_scale']]

#종속변수 저장
y = train['completed']


In [57]:
X.select_dtypes(include="object").columns

Index(['major type', 'major1_1', 'major1_2', 'job', 're_registration',
       'nationality', 'inflow_route', 'whyBDA', 'what_to_gain',
       'hope_for_group', 'previous_class_3', 'previous_class_4',
       'previous_class_5', 'previous_class_6', 'previous_class_7',
       'previous_class_8', 'desired_career_path', 'project_type',
       'incumbents_level', 'incumbents_lecture', 'incumbents_company_level',
       'incumbents_lecture_type', 'incumbents_lecture_scale'],
      dtype='object')

In [83]:
# X에서 오브젝트 데이터 타입 컬럼들만 뽑아서 라벨인코딩 시키기
for col in X.select_dtypes(include = 'object').columns:
    le = LabelEncoder()
    combined = pd.concat([X[col], X_test[col]], axis=0)
    le.fit(combined)
    
    X[col] = le.transform(X[col])
    X_test[col] = le.transform(X_test[col])


In [84]:

# major_data값 bool에서 정수로 바꾸기
X["major_data"]= X["major_data"].astype(np.int64)
X_test["major_data"]= X_test["major_data"].astype(np.int64)

In [60]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   generation                748 non-null    int64  
 1   school1                   748 non-null    int64  
 2   major_data                748 non-null    int64  
 3   class1                    748 non-null    int64  
 4   class2                    169 non-null    float64
 5   completed_semester        720 non-null    float64
 6   time_input                748 non-null    float64
 7   major type                748 non-null    int64  
 8   major1_1                  748 non-null    int64  
 9   major1_2                  748 non-null    int64  
 10  job                       748 non-null    int64  
 11  re_registration           748 non-null    int64  
 12  nationality               748 non-null    int64  
 13  inflow_route              748 non-null    int64  
 14  whyBDA    

In [61]:

# 독립변수 상위 세줄만 뽑아서 값 확인
X.head(3)

,generation,school1,major_data,class1,class2,completed_semester,time_input,major type,major1_1,major1_2,...,previous_class_6,previous_class_7,previous_class_8,desired_career_path,project_type,incumbents_level,incumbents_lecture,incumbents_company_level,incumbents_lecture_type,incumbents_lecture_scale
0,9,22,0,1,4.0,4.0,2.0,2,35,137,...,10,13,16,0,1,0,0,25,1,3
1,9,1,1,8,NaN,6.0,3.0,2,228,12,...,10,13,16,3,1,1,7,9,0,7
2,9,27,0,7,NaN,8.0,10.0,0,202,163,...,10,13,16,0,0,1,5,6,0,7


In [62]:
# Nan값 나머지랑 평균치내서 채워넣기
X.fillna(X.mean(),inplace=True)

In [63]:
# 마지막으로 트레이닝 준비가 완료되었는지 확인하기
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   generation                748 non-null    int64  
 1   school1                   748 non-null    int64  
 2   major_data                748 non-null    int64  
 3   class1                    748 non-null    int64  
 4   class2                    748 non-null    float64
 5   completed_semester        748 non-null    float64
 6   time_input                748 non-null    float64
 7   major type                748 non-null    int64  
 8   major1_1                  748 non-null    int64  
 9   major1_2                  748 non-null    int64  
 10  job                       748 non-null    int64  
 11  re_registration           748 non-null    int64  
 12  nationality               748 non-null    int64  
 13  inflow_route              748 non-null    int64  
 14  whyBDA    

In [64]:
# 트레이닝 테스트 데이터 나누기
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=49)

In [85]:
X_test.fillna(X_test.mean(),inplace=True)

In [86]:
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_valid = ss.transform(X_valid)
X_test = ss.transform(X_test)

In [66]:
from sklearn.model_selection import cross_val_score, KFold


def NB_objective(trial):

    var_smoothing = trial.suggest_float('var_smoothing', 1e-9, 1e-2, log=True)
    
    
    model = GaussianNB(var_smoothing=var_smoothing)

    score = cross_val_score(model, X, y, cv=10, scoring='f1_macro', n_jobs=-1).mean()
    
    return score

In [67]:
NB_study = optuna.create_study(direction='maximize')
NB_study.optimize(NB_objective, n_trials=100)

[I 2026-02-03 08:18:40,828] A new study created in memory with name: no-name-78d3d94b-198d-4274-97dc-b46bb859f5ae
[I 2026-02-03 08:18:40,914] Trial 0 finished with value: 0.2636873511825468 and parameters: {'var_smoothing': 4.437436592081534e-06}. Best is trial 0 with value: 0.2636873511825468.
[I 2026-02-03 08:18:40,942] Trial 1 finished with value: 0.23590787689646303 and parameters: {'var_smoothing': 0.0018997402929934175}. Best is trial 0 with value: 0.2636873511825468.
[I 2026-02-03 08:18:40,967] Trial 2 finished with value: 0.23590787689646303 and parameters: {'var_smoothing': 0.008746109394002128}. Best is trial 0 with value: 0.2636873511825468.
[I 2026-02-03 08:18:40,980] Trial 3 finished with value: 0.25984622296110743 and parameters: {'var_smoothing': 4.3113524688788015e-07}. Best is trial 0 with value: 0.2636873511825468.
[I 2026-02-03 08:18:41,005] Trial 4 finished with value: 0.27103848061511177 and parameters: {'var_smoothing': 1.2838865796880078e-08}. Best is trial 4 wit

In [68]:
print('Best parameter: ', NB_study.best_params)

Best parameter:  {'var_smoothing': 4.479943555707773e-09}


In [69]:
def objectiveXGB(trial):
    N_ESTIMATORS = trial.suggest_int('n_estimators', 100, 5000)
    MAX_DEPTH = trial.suggest_int('max_depth', 3, 50)
    LEARNING_RATE = trial.suggest_float('learning_rate', 0.01, 0.3)
    SUBSAMPLE = trial.suggest_float('subsample', 0.5, 1.0)
    COLSAMPLE_BYTREE = trial.suggest_float('colsample_bytree', 0.5, 1.0)
    REG_ALPHA = trial.suggest_float('reg_alpha', 1e-5, 1.0, log=True)
    REG_LAMBDA = trial.suggest_float('reg_lambda', 1e-5, 1.0, log=True)

    # 모델 정의
    model_xgb = XGBClassifier(
        n_estimators=N_ESTIMATORS,
        max_depth=MAX_DEPTH,
        learning_rate=LEARNING_RATE,
        subsample=SUBSAMPLE,
        colsample_bytree=COLSAMPLE_BYTREE,
        reg_alpha=REG_ALPHA,
        reg_lambda=REG_LAMBDA,
        eval_metric='logloss',
        use_label_encoder=False,
        random_state=42
    )
    score = cross_val_score(model_xgb, X, y, n_jobs=-1, cv=10, scoring='f1_macro')
    f1_macro = np.mean(score)

    return f1_macro

In [70]:
# def objectiveKNN(trial):
#     n_neighbors = trial.suggest_int('n_neighbors', 1, 30)
#     metric = trial.suggest_categorical('metric', ['euclidean', 'manhattan', 'minkowski'])
#     weights = trial.suggest_categorical('weights', ['uniform', 'distance'])
#     model = KNeighborsClassifier(n_neighbors=n_neighbors, metric=metric, weights=weights)
#     model.fit(X_train, y_train)
#     y_pred = model.predict(X_test)
#     return f1_score(y_test, y_pred)

In [71]:
def objectiveDT(trial):
    # Expanded hyperparameter search space
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    splitter = trial.suggest_categorical('splitter', ['best', 'random'])
    max_depth = trial.suggest_int('max_depth', 3, 50, step=3)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
    min_impurity_decrease = trial.suggest_float('min_impurity_decrease', 0.0, 0.1)
    ccp_alpha = trial.suggest_float('ccp_alpha', 0.0, 0.05)

    # Define model with sampled hyperparameters
    model = DecisionTreeClassifier(
        criterion=criterion,
        splitter=splitter,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        min_impurity_decrease=min_impurity_decrease,
        ccp_alpha=ccp_alpha,
        random_state=42
    )
    score = cross_val_score(model, X, y, n_jobs=-1, cv=10, scoring='f1_macro')
    f1_macro = np.mean(score)

    return f1_macro


In [72]:
# def objectiveLGB(trial):
#     param = {
#         'max_depth': trial.suggest_int('max_depth',3, 15),
#         'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 1e-2),
#         'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
#         'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
#         'subsample': trial.suggest_loguniform('subsample', 0.4, 1),
#     }
#     model_lgb = LGBMClassifier(**param)
#     model_lgb.fit(X_train, y_train)
#     y_pred = model_lgb.predict(X_test)
#     return f1_score(y_test, y_pred)

In [73]:
XGB_study = optuna.create_study(direction='maximize')
XGB_study.optimize(objectiveXGB, n_trials=100)

[I 2026-02-03 08:18:43,333] A new study created in memory with name: no-name-9439a43e-fa85-48e3-9dba-f9267157a052
[I 2026-02-03 08:18:43,500] Trial 0 finished with value: 0.5436346618092961 and parameters: {'n_estimators': 298, 'max_depth': 11, 'learning_rate': 0.18018650082813853, 'subsample': 0.8658399699798286, 'colsample_bytree': 0.9140089844228891, 'reg_alpha': 1.7002358687851082e-05, 'reg_lambda': 0.00010040009224978588}. Best is trial 0 with value: 0.5436346618092961.
[I 2026-02-03 08:18:44,167] Trial 1 finished with value: 0.4987267004261158 and parameters: {'n_estimators': 4374, 'max_depth': 42, 'learning_rate': 0.12129045712423385, 'subsample': 0.5495861966371864, 'colsample_bytree': 0.6906276648032685, 'reg_alpha': 0.000721491426820094, 'reg_lambda': 0.0007224514419666703}. Best is trial 0 with value: 0.5436346618092961.
[I 2026-02-03 08:18:44,448] Trial 2 finished with value: 0.5146810182685794 and parameters: {'n_estimators': 1878, 'max_depth': 5, 'learning_rate': 0.208154

In [74]:
DT_study = optuna.create_study(direction='maximize')
DT_study.optimize(objectiveDT, n_trials=100)

[I 2026-02-03 08:19:15,673] A new study created in memory with name: no-name-78980830-09d2-41d7-8bc3-93901c6c1386
[I 2026-02-03 08:19:15,700] Trial 0 finished with value: 0.4124055782089739 and parameters: {'criterion': 'entropy', 'splitter': 'random', 'max_depth': 30, 'min_samples_split': 20, 'min_samples_leaf': 10, 'max_features': None, 'min_impurity_decrease': 0.0764441089850042, 'ccp_alpha': 0.03759072524661001}. Best is trial 0 with value: 0.4124055782089739.
[I 2026-02-03 08:19:15,727] Trial 1 finished with value: 0.4124055782089739 and parameters: {'criterion': 'entropy', 'splitter': 'best', 'max_depth': 21, 'min_samples_split': 8, 'min_samples_leaf': 2, 'max_features': 'log2', 'min_impurity_decrease': 0.07719987895453724, 'ccp_alpha': 0.018618312304604933}. Best is trial 0 with value: 0.4124055782089739.
[I 2026-02-03 08:19:15,755] Trial 2 finished with value: 0.4124055782089739 and parameters: {'criterion': 'entropy', 'splitter': 'random', 'max_depth': 48, 'min_samples_split':

In [75]:
# LGBM_study = optuna.create_study(direction='maximize')
# LGBM_study.optimize(objectiveLGB, n_trials=100)

In [76]:
# KNN_study = optuna.create_study(direction='maximize')
# KNN_study.optimize(objectiveKNN, n_trials=100)

In [77]:
print("Naive Bayes: ", NB_study.best_params)
print('xgboost: ', XGB_study.best_params)
print('DT: ', DT_study.best_params)

Naive Bayes:  {'var_smoothing': 4.479943555707773e-09}
xgboost:  {'n_estimators': 3229, 'max_depth': 16, 'learning_rate': 0.2914537044143241, 'subsample': 0.8501234371186468, 'colsample_bytree': 0.5874920859968593, 'reg_alpha': 1.3695489548351062e-05, 'reg_lambda': 0.0003561382613365196}
DT:  {'criterion': 'gini', 'splitter': 'best', 'max_depth': 45, 'min_samples_split': 3, 'min_samples_leaf': 9, 'max_features': None, 'min_impurity_decrease': 0.0005167238103026932, 'ccp_alpha': 8.233219802367825e-05}


In [78]:
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

base_models = [
    ('rf_1', GaussianNB(**NB_study.best_params)),
    ('rf_2', DecisionTreeClassifier(**DT_study.best_params))
]

clf = StackingClassifier(estimators=base_models, \
                   final_estimator=XGBClassifier(**XGB_study.best_params))

clf.fit(X_train, y_train)

StackingClassifier(cv=None,
                   estimators=[('rf_1',
                                GaussianNB(priors=None,
                                           var_smoothing=4.479943555707773e-09)),
                               ('rf_2',
                                DecisionTreeClassifier(ccp_alpha=8.233219802367825e-05,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=45,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0005167238103026932,
                                                       min_samples_leaf=9,
                                                       min_samples_split=3,
                                                       min_weight...
                                                 interaction_constraints=None,
                                                 learning_rate=0.2914537044143241,
                                                 max_bin=None,
                                                 max_cat_threshold=None,
                                                 max_cat_to_onehot=None,
                                                 max_delta_step=None,
                                                 max_depth=16, max_leaves=None,
                                                 min_child_weight=None,
                                                 missing=nan,
                                                 monotone_constraints=None,
                                                 multi_strategy=None,
                                                 n_estimators=3229, n_jobs=None,
                                                 num_parallel_tree=None, ...),
                   n_jobs=None, passthrough=False, stack_method='auto',
                   verbose=0)

In [89]:
pred = clf.predict(X_test)

submission = pd.read_csv('/Users/euijinlee/Downloads/open/sample_submission.csv')
submission['completed'] = pred
submission.to_csv('submit.csv', index = False)